## 1、环境配置

In [ ]:
# 克隆paddleSeg的github仓库
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git
!pip install -r PaddleSeg/requirements.txt

## 2、数据准备

In [ ]:
# 解压数据集
!mkdir ~/PaddleSeg/datasets
!unzip -q data/data77571/train_and_label.zip -d ~/PaddleSeg/datasets
!unzip -q data/data77571/img_test.zip -d ~/PaddleSeg/datasets

## 3、数据预处理

### （1）数据集划分模块

In [ ]:
import numpy as np
import os
os.chdir('/home/aistudio/PaddleSeg/datasets/')

datas = []
image_base = 'img_train'   # 训练集原图路径
annos_base = 'lab_train'   # 训练集标签路径

# 读取原图文件名
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

# 将训练集的图像集和标签路径写入datas中
for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    datas.append((img_pt0.replace('/home/aistudio', ''), img_pt1.replace('/home/aistudio', '')))
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):
        pass
    else:
        raise "path invalid!"

# 打印datas的长度和具体存储例子
print('total:', len(datas))
print(datas[0][0])
print(datas[0][1])
print(datas[10][:])

### （2）划分数据集

In [ ]:
import cv2
import numpy as np
import os
os.chdir('/home/aistudio/PaddleSeg/datasets/')
# 四类标签，这里用处不大，比赛评测是以0、1、2、3类来对比评测的
labels = ['建筑', '耕地', '林地', '其他']

# 将labels写入标签文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')

# 随机打乱datas
np.random.seed(3407)
np.random.shuffle(datas)

# 验证集与训练集的划分，0.05表示5%为验证集，95%为训练集
split_num = int(0.05*len(datas))

# 划分训练集和验证集
train_data = datas[:-split_num]
valid_data = datas[-split_num:]

# 写入训练集list
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')

# 写入验证集list
with open('val_list.txt', 'w') as f:
    for img, lbl in valid_data:
        # 进行数据清洗，数据验证过程中，对于全为255的图像直接忽略
        clean = cv2.imread(lbl)
        if (clean == 255).all():
            continue
        f.write(img + ' ' + lbl + '\n')

# 打印训练集和测试集大小
print('train:', len(train_data))
print('valid:', len(valid_data))

### （3）分析数据类别样本数量

In [ ]:
import cv2
import numpy as np

NUM_CLASSES = 4

area = {i : 0 for i in range(NUM_CLASSES)}
area_proportion = {i : {0 : 0, 1 : 0, 2 : 0, 3 : 0} for i in range(NUM_CLASSES)}
area[255] = 0
image_num = 0

def calc(image, num_classes=NUM_CLASSES):
    label_image = np.array(image)
    for cls in range(num_classes):
        area[cls] += np.count_nonzero(label_image == cls)
    area[255] += np.count_nonzero(label_image == 255)

def area_calc(image, num_classes=NUM_CLASSES):
    label_image = np.array(image)
    image_area = label_image.shape[0] * label_image.shape[1]
    for cls in range(num_classes):
        proportion = np.count_nonzero(label_image == cls) / float(image_area)
        if proportion < 0.01:
            area_proportion[cls][0] += 1
        elif proportion < 0.02:
            area_proportion[cls][1] += 1
        elif proportion < 0.8:
            area_proportion[cls][2] += 1
        else:
            area_proportion[cls][3] += 1


# 统计四种类型的面积占比
train_file_dir = '/home/aistudio/PaddleSeg/datasets/train_list.txt'
val_file_dir = '/home/aistudio/PaddleSeg/datasets/val_list.txt'
with open(train_file_dir, 'r') as f:
    for line in f.readlines():
        if image_num % 5000 == 0:
            print("当前已读取"+str(image_num)+"个样本，进度为 "+str(100*image_num/len(os.listdir("img_train"))/0.95)+"%")
        label_dir = line.split()[1]
        image_label = cv2.imread(label_dir, cv2.IMREAD_GRAYSCALE)
        calc(image_label)
        area_calc(image_label)
        image_num += 1


for cls in range(NUM_CLASSES):
    area[cls] = area[cls] / (image_num * 256.0 * 256.0)
area[255] = area[255] / (image_num * 256.0 * 256.0)
print(area)
print(area_proportion)


#参考https://aistudio.baidu.com/aistudio/projectdetail/4556036

![](https://ai-studio-static-online.cdn.bcebos.com/ee21bcfab01c49e3a087e4e097f4d4e9fcf80f73f0df4e79ba9682196d28bb65)

### （4）对已划分的数据集进行重采样
**在训练过程中发现第三类mIoU明显低于其他类别，并且发现第一类和第三类非常相似，考虑对第三类进行重采样，并进行额外的数据增强。**

In [ ]:
import cv2
import numpy as np
import os
import shutil
os.chdir('/home/aistudio/PaddleSeg/datasets/')

PROB = 0.1  #重采样概率

with open('train_list.txt', 'r') as f:  #把txt写入list中
    lines = f.readlines()

#如果发现类别3存在于img中，那么把该图像的位置在train_list里复制一次
with open('train_list.txt', 'a+') as f:
    index = 0
    count = 0
    samp_count = 0
    for line in lines:
        if index % 5000 == 0:
            print("当前已读取"+str(index)+"个样本，进度为 "+str(100*index/len(lines))+"%")
        label_dir = line.split()[1]
        train_dir = line.split()[0]
        image_label = cv2.imread(label_dir, cv2.IMREAD_GRAYSCALE)
        if 3 in image_label[0]:
            img = cv2.imread(train_dir, cv2.IMREAD_COLOR)
            count += 1
            if np.random.random()<PROB:
                f.write("\n"+"img_train/T{}.jpg".format("RE"+str(index))+' '+"lab_train/T{}.png".format("RE"+str(index)))
                rand_rotate = np.random.randint(0,3)
                img_t = cv2.rotate(cv2.bilateralFilter(img, 9, 75, 75), rand_rotate) #随机旋转并且进行双边模糊
                lab_t = cv2.rotate(image_label, rand_rotate)
                cv2.imwrite("img_train/T{}.jpg".format("RE"+str(index)),img_t)
                cv2.imwrite("lab_train/T{}.png".format("RE"+str(index)),lab_t)
                samp_count += 1
        index += 1

print("对含有该类的图像数量为：", count)
print("对含有该类的图像并进行重采样的数量为：", samp_count)

**第二类和第三类的像素点数量都一样少，但第三类的mIoU更低
猜想可能是数量最多第一类影响了第三类的mIoU**

**若同时对第二类进行重采样，mIoU略微下降**

![](https://ai-studio-static-online.cdn.bcebos.com/29b2de03209e4d8e9efcf654d5f6e31535ee14220425445391199fe40ca378b3)


## 5、模型训练

![](https://ai-studio-static-online.cdn.bcebos.com/d3da1d6ef87945a287e71e8287134bcfe5542dee7139402cb5f602097ba5ad54)


In [ ]:
%cd ~/PaddleSeg
! python train.py \
       --config configs/segformer/segformer_b2_cityscapes_1024x1024_160k.yml \  #使用的训练配置
       --save_interval 2000 \  #每多少iterations保存一次模型
       --use_vdl \  #保存可视化数据
       --log_iters 50 \  #每多少iterations打印一次当前训练信息
       --save_dir output/special \  #保存地址
       --do_eval \  #每次模型保存前进行一次评估
#       --resume_model output/special/iter_10000 \  #继续训练模型

![](https://ai-studio-static-online.cdn.bcebos.com/13bf511388ef49a483c81b1bfd7377346765b873a93e462b99764b181553f893)
![](https://ai-studio-static-online.cdn.bcebos.com/78d828460d4c4871bdb542d950630af8b4d73e777fca4dcda4eedaf05d0e11d8)


### **逃离鞍点的方法**
**测试出造成梯度爆炸最低学习率lr1（50iters内loss＞5），将模型学习率调整至0.8*lr1训练直到loss大于原先loss的2倍后恢复正常训练。**

## 6、模型评估

In [ ]:
# 模型评估
%cd ~/PaddleSeg
! python val.py \
       --config configs/segformer/segformer_b2_cityscapes_1024x1024_160k.yml \
       --model_path output/resegformer/iter_480000/model.pdparams

![](https://ai-studio-static-online.cdn.bcebos.com/5691eeaecda0442ea37c877469ab242e10d3fe7974ce45ffaa5794afced523f6)


## 7、测试结果

In [ ]:
# 测试集预测
%cd ~/PaddleSeg
!python predict.py \
       --config configs/segformer/segformer_b2_cityscapes_1024x1024_160k.yml \
       --model_path output/special/iter_608000/model.pdparams \
       --image_path datasets/img_testA \
       --save_dir result

In [ ]:
# 由预测结果生成提交文件
%cd ~/PaddleSeg
!zip -r result.zip result/

![](https://ai-studio-static-online.cdn.bcebos.com/5bc884da0d514ec3902c21a4dfebffb8d581edd09ceb42219ec5c8ea6f4e04f0)


## 8、基于结果的多模型融合（目前仍存bug）

### **result评分时异常，但可能会涨点**

In [ ]:
!mkdir ~/work/results/merge_result
!mkdir ~/work/results/merge_result/pseudo_color_prediction
!mkdir ~/work/results/merge_result/added_prediction

In [ ]:
import cv2
import numpy as np
import os
import shutil
os.chdir('/home/aistudio/work/results/')

p = 0.1
index = 0

for f in os.listdir("result-segformerb2-65.09/added_prediction"):

    # time_start = time.clock()
    
    if index % 10 == 0:
        print("当前已读取"+str(index)+"个样本，进度为 "+str(100*index/len(os.listdir("result-segformerb2-65.09/added_prediction")))+"%")
    file_name = f.split(".")[0]
    color_map1 = cv2.imread("result-segformerb2-65.09/pseudo_color_prediction/{}.png".format(file_name), cv2.IMREAD_COLOR)
    color_map2 = cv2.imread("result-segformerb2-66.24/pseudo_color_prediction/{}.png".format(file_name), cv2.IMREAD_COLOR)
    color_map3 = cv2.imread("result-segformerb3/pseudo_color_prediction/{}.png".format(file_name), cv2.IMREAD_COLOR)
    add1 = cv2.imread("result-segformerb2-65.09/added_prediction/{}.jpg".format(file_name), cv2.IMREAD_COLOR)
    add2 = cv2.imread("result-segformerb2-66.24/added_prediction/{}.jpg".format(file_name), cv2.IMREAD_COLOR)
    add3 = cv2.imread("result-segformerb3/added_prediction/{}.jpg".format(file_name), cv2.IMREAD_COLOR)

    for i in range(256):
        for j in range(256):
            if (color_map2[i][j] == color_map3[i][j]).all():
                color_map1[i][j] = color_map2[i][j]
                add1[i][j] = add2[i][j]

    cv2.imwrite("merge_result/pseudo_color_prediction/{}.png".format(file_name).format("RE"+str(index)), color_map1)
    cv2.imwrite("merge_result/added_prediction/{}.jpg".format(file_name), add1)

    index += 1

    # time_end = time.clock()
    # tim = time_end - time_start
    # minute = tim % 60
    # sec = tim - 60*minute
    # print("ETA: {}:{}".format(minute, sec))

## 9、总结

![](https://ai-studio-static-online.cdn.bcebos.com/29b829a18a524f288d960b99b87fe027112ebafd530945d3bc5b3346a1cd2ff5)


### 项目作者

> 赵祎安 大连理工大学 计算机科学与技术 2019级 大工飞桨领航团团长

> modified by Friman 吴玮彤 大连理工大学 电子信息类（创新班） 2022级